# 0. Outline

1. 生成模型 vs. 判别模型

2. LR和最大似然

3. L1 vs. L2 正则

4. 交叉验证

5. 正则的灵活使用

6. MLE vs. MAP

7. 特征选择与LASSO

8. 坐标下降法

# 1. 生成模型 vs. 判别模型

1. 字面上的区别

$$
\begin{array}
\text{Generative Model:} \quad \boxed{Data} \rightarrow \boxed{Model} \rightarrow \boxed{New ~ Data}\\
\text{Discriminate Model:} \quad \boxed{Data} \rightarrow \boxed{Model} \rightarrow \boxed{判别}\\
\end{array}
$$

生成模型：Naive Bayes, HMM, VAE, GAN

判别模型：LR, CRF

2. 训练模型的目标函数

生成： Maximize $P(X,Y)$

判别： Maximize $P(Y|X)$

3. 生成模型生成数据的原理

生成: $P(x,y)=\color{red}{P(x|y)}\cdot P(y)$

判别: $P(y|x)$

其中，$\color{red}{P(x|y)}$反应了数据的分布情况，从中采样可得到新的样本。

4. 生成模型用于判别

生成: $P(x,y)=P(y|x)P(x)$  

判别: $P(y|x)$

对于分类问题，通常判别模型要优于生成模型；而当数据量少时，生成模型可能优于判别模型（因为$P(x)$是先验知识）

# 2. Logistic Regression

$$
\begin{eqnarray}
P(y=1|x) &=& \sigma(w^{\mathrm{T}}x + b )  \tag{1}\\ 
P(y=0|x) &=& 1-\sigma(w^{\mathrm{T}}x + b ) \tag{2}
\end{eqnarray}
$$

可改写成

$$
P(y|x,w)=P(y=1|x,w)^{y}\left[1-P(y=1|x,w)\right]^{1-y} \tag{3}
$$

参数$w,b$可通过MLE进行估计:

$$
\hat{w}, \hat{b} = \arg \max_{w,b} \prod_{i=1}^{n}p(y_{i} | x_{i}; w, b) \tag{4}
$$

$$
\begin{eqnarray}
\arg \min &~& -\sum_{i=1}^{n} \log p(y_{i}|x_{i};w,b) \\
&=& -\sum_{i=1}^{n} \log \left[ P(y_{i}=1|x_{i},w,b)^{y_{i}}\left[1-P(y_{i}=1|x_{i},w,b)\right]^{1-y_{i}} \right] \\
&=& -\sum_{i=1}^{n} y_{i} \log \sigma(w^{\mathrm{T}}x_{i}+b) + (1-y_{i}) \log \left[1- \sigma(w^{\mathrm{T}}x_{i}+b)\right] \\
&=& L(w,b) \tag{5}
\end{eqnarray}
$$

分别对$w,x$计算偏导

$$
\begin{eqnarray}
\frac{\partial L(w,b)}{\partial w} &=& \sum_{i=1}^{n} \left[\sigma(w^{\mathrm{T}}x_{i}+b) - y_{i} \right] \cdot x_{i} \tag{6} \\
\frac{\partial L(w,b)}{\partial b} &=& \sum_{i=1}^{n} \left[\sigma(w^{\mathrm{T}}x_{i}+b) - y_{i} \right] \tag{7}
\end{eqnarray}
$$

可使用梯度下降法或拟牛顿法求出$~\hat{w}, \hat{b}$

$$
\begin{array}{l}
for \quad t=1,2, \cdots, converge: \\
\quad \quad w^{(t)} = w^{(t-1)} - \eta_{t}\cdot \nabla_{w}L(w,b)\\
\quad \quad b^{(t)} = b^{(t-1)} - \eta_{t}\cdot \nabla_{b}L(w,b)
\end{array}
$$

- Empirical Gradient: 式(6)(7)中用样本计算的梯度 

- True Gradient


- GD: 遍历所有样本计算gradient（比较慢）

- SGD: 每次随机选取one sample计算gradient

- Mini-Batch: 使用部分样本计算gradient（推荐）

# 3. 正则化的LR

#### 1. Overfitting
- More Data
- Regularization
- Ensemble
- less complicated model
- less feature
- add noise

#### 2. 范数

若$w$是向量:

1-范数: $\|w \|_{1} = \sum_{i=1}^{d}|w_{i}|$ 

2-范数: $\|w \|_{2} = \sqrt{\sum_{i=1}^{d}w_{i}^{2}}$

若$w$是矩阵:

1-范数: $\|w\|_{1} = \max_{j}\sum_{i=1}^{m}|w_{ij}|$

2-范数: $\|w\|_{2} = \sqrt{\lambda_{1}}$

F-范数: $\|w\|_{F} = \left(\sum_{i=1}^{m}\sum_{j=1}^{n} w_{ij}^{2}\right)^{1/2}$

L2正则，指向量的2-范数，或矩阵的F-范数。

L1正则的Logistic Regression: （Lasso）

$$
\hat{w}, \hat{b} = \arg \min_{w,b} ~ -\sum_{i=1}^{n} \log p(y_{i}|x_{i};w,b) + \lambda \|w \|_{1} \\
$$

L2正则的Logistic Regression: （Ridge Regression）

$$
\hat{w}, \hat{b} = \arg \min_{w,b} ~ -\sum_{i=1}^{n} \log p(y_{i}|x_{i};w,b) + \lambda \|w \|_{2}^{\color{red}{2}} \\
$$

#### 3. L1 vs. L2

都能使得$~w~$变小，但L1正则学得的$~w~$是稀疏的，可以用于特征选择。

比如：

L1正则学得的$w=(0, ~0,~ 0.2,~ 0,~ \cdots,~ 0.7)$

L2正则学得的$w=(0.01, ~0.02,~ 0.2,~ 0,~ \cdots,~ 0.7)$

假设$w=(w_{1}, w_{2})\in \mathbf{R}^{2}$

$$
\begin{align}
f(x) &= L(w,b) + \lambda \|w\|_{1} \\
f(x) &= L(w,b) + \lambda \|w\|_{2}^{2}
\end{align}
$$   

<talbe>
    <td><img src='attachment/06. L1正则.png' style='zoom:50%'/></td>
    <td><img src='attachment/06. L2正则.png' style='zoom:50%'/></td>
</talbe>

从图中可以看出，当$w$的维度较高时，L1正则的loss与regularization有较大概率在0点相交。而L2正则相交到0点的概率则小了很多。

L1正则特征稀疏的好处:  

1. 如果维度太高，计算量也变得很高

2. 在稀疏条件下，计算量只依赖非0项

3. 提高可解释性

# 4. Lasso（L1正则的Linear Regression）

$$
L(w,b) = \sum_{i=1}^{n} \left(\sum_{j=1}^{d}w_{j}x_{ij} +  b - y_{i} \right)^{2} + \lambda \sum_{j=1}^{d}|w_{j}|
$$

以计算$w_{l}$的最优值为例说明为什么L1正则引起稀疏解。

$$
\begin{align}
\frac{\partial L}{\partial w_{l}} &= 2 \sum_{i=1}^{n} \left(\sum_{j=1}^{d}w_{j}x_{ij} +  b - y_{i} \right) \cdot x_{ij} + \lambda \frac{\partial |w_{l}|}{\partial w_{l}} \\
&= 2 \sum_{i=1}^{n} \left(\sum_{j \neq 1}^{d}w_{j}x_{ij} +  b - y_{i} + w_{l}x_{il} \right) \cdot x_{ij} + \lambda \frac{\partial |w_{l}|}{\partial w_{l}} \\
&= 2 \sum_{i=1}^{n} \left(\sum_{j \neq l}^{d}w_{j}x_{ij} +  b - y_{i}  \right) \cdot x_{ij} + 2\sum_{i=1}^{n} w_{l}x_{il}^{2} + \lambda \frac{\partial |w_{l}|}{\partial w_{l}} \\
&= 2 \sum_{i=1}^{n} \left(\sum_{j \neq l}^{d}w_{j}x_{ij} +  b - y_{i}  \right) \cdot x_{ij} + w_{l}\cdot \sum_{i=1}^{n} 2x_{il}^{2} + \lambda \frac{\partial |w_{l}|}{\partial w_{l}} \\
&= C_{l} + w_{l}\cdot a_{l} + \lambda \frac{\partial |w_{l}|}{\partial w_{l}} \\
&= \left\{\begin{matrix}
 C_{l} + w_{l}\cdot a_{l} + \lambda , & w_{l} > 0\\
 \left[C_{l} - \lambda, C_{l} + \lambda \right], & w_{l} > 0 \\
 C_{l} + w_{l}\cdot a_{l} -  \lambda, & w_{l} < 0
\end{matrix}\right.
\end{align} 
$$

令$\frac{\partial L}{\partial w_{l}} = 0$

当$w_{l}>0$时: 

$$
C_{l} + w_{l}\cdot a_{l} + \lambda = 0 \\
\color{blue}{w_{l} = \frac{-C_{l}-\lambda}{a_{l}}} \\
\because a_{l} = \sum_{i=1}^{n}2x_{il}^{2} > 0 \\
\therefore -C_{l}-\lambda > 0 \\
\therefore  \color{blue}{C_{l} < - \lambda}
$$

当$w_{l}<0$时: 

$$
C_{l} + w_{l}\cdot a_{l} - \lambda = 0 \\
\color{blue}{w_{l} = \frac{-C_{l} + \lambda}{a_{l}}} \\
\because a_{l} = \sum_{i=1}^{n}2x_{il}^{2} > 0 \\
\therefore -C_{l} + \lambda < 0 \\
\therefore \color{blue}{C_{l} >  \lambda}
$$

最终得到$w_{l}$的最优解为: 

$$
\hat{w_{l}} = \left\{\begin{matrix}
 \frac{-C_{l}-\lambda}{a_{l}}, & if \quad C_{l} < -\lambda \\
 \color{red}{0}, & if \color{red}{\quad -\lambda \leq C_{l} \leq \lambda} \Rightarrow \color{red}{sparcity} \\
\frac{-C_{l} + \lambda}{a_{l}} , & if \quad C_{l} > \lambda
\end{matrix}\right.
$$

# 5. Ridge（L2正则的Linear Regression）

$$
L = \sum_{i=1}^{n} \left(w^{\mathrm{T}}x_{i} - y_{i} \right)^{2} + \lambda \|w \|_{2}^{2}
$$

不带正则的Linear Regression的解为 $w=(x^{\mathrm{T}}x)^{-1}x^{\mathrm{T}}y$

L2正则的Linear Regression的解为 $w=(x^{\mathrm{T}}x + \lambda I)^{-1}x^{\mathrm{T}}y$